In [1]:
!pip install torch_geometric

In [2]:
import torch
import torch_geometric
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv

In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

In [4]:
print(f"Dataset: {dataset}:")
print('Number of graphs:', len(dataset))
print('Number of features:', dataset.num_node_features)
print('Number of classes:', dataset.num_classes)


print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Is the graph directed: {data.is_directed()}")
print(f"Node feature matrix size: {data.x.size()}")
print(f"Labels size: {data.y.size()}")

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7
Number of nodes: 2708
Number of edges: 10556
Is the graph directed: False
Node feature matrix size: torch.Size([2708, 1433])
Labels size: torch.Size([2708])


In [5]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
model = GCN(dataset.num_node_features, hidden_channels=16, num_classes=dataset.num_classes).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [8]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

In [9]:
epochs = 200
for epoch in range(epochs):
    loss = train()
    if epoch % 10 == 0:
        test_acc = test()
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')

Epoch 0, Loss: 1.9458, Test Accuracy: 0.2400
Epoch 10, Loss: 1.8561, Test Accuracy: 0.7080
Epoch 20, Loss: 1.7051, Test Accuracy: 0.7390
Epoch 30, Loss: 1.5241, Test Accuracy: 0.7570
Epoch 40, Loss: 1.3129, Test Accuracy: 0.7810
Epoch 50, Loss: 1.0950, Test Accuracy: 0.7830
Epoch 60, Loss: 0.9160, Test Accuracy: 0.7900
Epoch 70, Loss: 0.7674, Test Accuracy: 0.8060
Epoch 80, Loss: 0.7126, Test Accuracy: 0.7920
Epoch 90, Loss: 0.6480, Test Accuracy: 0.7910
Epoch 100, Loss: 0.5482, Test Accuracy: 0.8010
Epoch 110, Loss: 0.5454, Test Accuracy: 0.7980
Epoch 120, Loss: 0.4529, Test Accuracy: 0.8010
Epoch 130, Loss: 0.4490, Test Accuracy: 0.8010
Epoch 140, Loss: 0.4161, Test Accuracy: 0.7950
Epoch 150, Loss: 0.4209, Test Accuracy: 0.7950
Epoch 160, Loss: 0.3646, Test Accuracy: 0.7890
Epoch 170, Loss: 0.3509, Test Accuracy: 0.7930
Epoch 180, Loss: 0.3380, Test Accuracy: 0.7900
Epoch 190, Loss: 0.3207, Test Accuracy: 0.8050
